# Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import os

# from .autonotebook import tqdm as notebook_tqdm
from sentence_transformers import SentenceTransformer

c:\Users\Yizhar\Desktop\cursor_projects\vi_home_assignment\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# EDA

In [15]:

data_path = "../data"

# # see what's there
# for file in os.listdir(f'{data_path}/train'):
#   print(file)
#   df = pd.read_csv(f'{data_path}/train/{file}')
#   display(df.sample(10))
#   # also analyze the categorical columns
#   for col in df.columns:
#     if df[col].dtype == 'object':
#       print(f'{col}: {df[col].nunique()}')

In [16]:
train_web_visits = pd.read_csv(f'{data_path}/train/web_visits.csv')
train_claims_df = pd.read_csv(f'{data_path}/train/claims.csv')
train_churn_labels_df = pd.read_csv(f'{data_path}/train/churn_labels.csv')
train_app_usage_df = pd.read_csv(f'{data_path}/train/app_usage.csv')

test_web_visits_df = pd.read_csv(f'{data_path}/test/test_web_visits.csv')
test_claims_df = pd.read_csv(f'{data_path}/test/test_claims.csv')
test_churn_labels_df = pd.read_csv(f'{data_path}/test/test_churn_labels.csv')
test_app_usage_df = pd.read_csv(f'{data_path}/test/test_app_usage.csv')

In [17]:

train_app_usage_df['session_date'] = train_app_usage_df['timestamp'].dt.date

app_sessions_per_member = (
    train_app_usage_df
    .groupby('member_id')
    .agg(
        num_app_sessions=('timestamp', 'size'),
        avg_daily_app_sessions=('timestamp', lambda x: x.count() / x.nunique() if x.nunique() > 0 else 0)
    )
    .reset_index()
)
app_sessions_per_member

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
# del embedding_model

In [67]:
# Make a row per member-icd_code
member_icd_counts = (
    train_claims_df.groupby(['member_id', 'icd_code'])
    .size()
    .reset_index(name='count')
)

# Calculate total claims per member
member_total_claims = (
    train_claims_df.groupby('member_id')
    .size()
    .rename('total_claims')
    .reset_index()
)

# Merge counts with total claims to get the ratio
member_icd_counts = member_icd_counts.merge(member_total_claims, on='member_id', how='left')
member_icd_counts['ratio'] = member_icd_counts['count'] / member_icd_counts['total_claims']

# Pivot such that for each member, for every icd_code there are two columns:
# 'icd_{code}_count' and 'icd_{code}_ratio'
icd_code_list = sorted(train_claims_df['icd_code'].unique())
# Build pivot tables for counts and ratios
counts_pivot = member_icd_counts.pivot(index='member_id', columns='icd_code', values='count')
ratios_pivot = member_icd_counts.pivot(index='member_id', columns='icd_code', values='ratio')

# Rename columns accordingly
counts_pivot.columns = [f'icd_{col}_count' for col in counts_pivot.columns]
ratios_pivot.columns = [f'icd_{col}_ratio' for col in ratios_pivot.columns]

# Concatenate both count and ratio columns
member_icd_counts_ratios_df = pd.concat([counts_pivot, ratios_pivot], axis=1).reset_index().fillna(0)

member_icd_counts_ratios_df

,member_id,icd_A09_count,icd_B34.9_count,icd_E11.9_count,icd_H10.9_count,icd_I10_count,icd_J00_count,icd_K21.9_count,icd_M54.5_count,icd_R51_count,...,icd_A09_ratio,icd_B34.9_ratio,icd_E11.9_ratio,icd_H10.9_ratio,icd_I10_ratio,icd_J00_ratio,icd_K21.9_ratio,icd_M54.5_ratio,icd_R51_ratio,icd_Z71.3_ratio
0,1,0.0,0.0,1.0,0.0,1.0,4.0,0.0,1.0,0.0,...,0.000000,0.000000,0.111111,0.000000,0.111111,0.444444,0.000000,0.111111,0.000000,0.222222
1,2,0.0,0.0,3.0,0.0,2.0,0.0,1.0,0.0,1.0,...,0.000000,0.000000,0.333333,0.000000,0.222222,0.000000,0.111111,0.000000,0.111111,0.222222
2,3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.250000,0.500000,0.000000,0.000000
3,4,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,...,0.200000,0.000000,0.000000,0.000000,0.600000,0.200000,0.000000,0.000000,0.000000,0.000000
4,5,0.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,0.0,...,0.000000,0.000000,0.285714,0.000000,0.142857,0.000000,0.142857,0.285714,0.000000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,9996,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.111111,0.222222,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.333333
9976,9997,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.200000,0.000000,0.200000,0.200000
9977,9998,0.0,2.0,0.0,2.0,0.0,4.0,2.0,1.0,1.0,...,0.000000,0.142857,0.000000,0.142857,0.000000,0.285714,0.142857,0.071429,0.071429,0.142857
9978,9999,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.333333,0.000000,0.500000


In [53]:


# Load the Qwen embedding model via sentence-transformers
# NOTE: Qwen3-Embedding-0.6B must be supported by your sentence-transformers installation
try: 
    embedding_model
except:
    embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

# Get embeddings for each topic in interests
interest_embeddings = embedding_model.encode(interests, show_progress_bar=True, truncate_dim=32)

# Create a DataFrame with each topic and its embedding vector
interest_embeddings_df = pd.DataFrame({
    'topic': interests,
    'embedding': list(interest_embeddings)
})

# Display the resulting dataframe (optional)
interest_embeddings_df.head()


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]


,topic,embedding
0,chronic,"[-0.02096963, -0.0020781416, -0.015947886, 0.0..."
1,tech,"[-0.03774623, -0.046034474, -0.012437088, -0.0..."
2,heart,"[-0.031568803, 0.055135973, -0.011493279, -0.0..."
3,gaming,"[0.011342987, -0.07814658, -0.011934398, -0.06..."
4,stress,"[0.00910086, 0.028227106, -0.011203192, 0.0461..."


In [56]:
# Explode the embedding column into multiple columns (one per embedding dimension)
embedding_dims = len(interest_embeddings_df.iloc[0]['embedding'])
embedding_cols = [f'interest_embedding_dim_{i}' for i in range(embedding_dims)]
embedding_df_expanded = pd.DataFrame(interest_embeddings_df['embedding'].tolist(), columns=embedding_cols)
interest_embeddings_exploded_df = pd.concat([interest_embeddings_df.drop(columns=['embedding']), embedding_df_expanded], axis=1)


In [57]:
interest_embeddings_exploded_df

,topic,interest_embedding_dim_0,interest_embedding_dim_1,interest_embedding_dim_2,interest_embedding_dim_3,interest_embedding_dim_4,interest_embedding_dim_5,interest_embedding_dim_6,interest_embedding_dim_7,interest_embedding_dim_8,...,interest_embedding_dim_22,interest_embedding_dim_23,interest_embedding_dim_24,interest_embedding_dim_25,interest_embedding_dim_26,interest_embedding_dim_27,interest_embedding_dim_28,interest_embedding_dim_29,interest_embedding_dim_30,interest_embedding_dim_31
0,chronic,-0.020970,-0.002078,-0.015948,0.024750,0.078295,0.009863,-0.015813,0.026154,-0.011166,...,-0.048722,0.120367,0.096834,0.039852,-0.016819,0.101452,0.034518,-0.036859,0.034069,-0.022365
1,tech,-0.037746,-0.046034,-0.012437,-0.099354,0.012120,-0.029332,0.046414,0.110775,-0.071351,...,-0.025287,0.164109,-0.050859,0.042931,-0.101809,0.120021,-0.040853,-0.026118,0.041702,-0.004998
2,heart,-0.031569,0.055136,-0.011493,-0.013442,0.019451,0.007551,-0.020874,0.000962,-0.021293,...,-0.066453,0.154941,0.065421,0.018841,-0.046219,0.117754,0.022808,-0.013405,0.020330,-0.017481
3,gaming,0.011343,-0.078147,-0.011934,-0.069321,0.097613,-0.047287,-0.031151,0.053877,0.033999,...,-0.060459,0.124194,-0.053141,-0.013678,-0.100397,0.098130,-0.009910,0.005994,-0.010224,0.035385
4,stress,0.009101,0.028227,-0.011203,0.046150,0.030723,-0.004414,-0.002363,0.014776,-0.056125,...,-0.082783,0.122195,0.066319,0.019669,-0.052947,0.124508,-0.009055,-0.011867,0.011851,-0.009912
5,sleep,-0.032511,-0.022964,-0.010948,-0.003653,0.020287,0.004675,-0.035857,0.059914,-0.087480,...,-0.085280,0.129373,0.055816,0.024948,-0.055685,0.069869,0.024755,-0.016870,0.010970,0.037278
6,wellness,-0.053759,0.013073,-0.012045,0.039326,0.043685,-0.032392,0.022982,0.031512,-0.000988,...,-0.045779,0.134382,-0.030601,0.010409,-0.034643,0.071397,0.018357,0.008129,0.046356,-0.012824
7,aerobic,-0.007625,-0.000393,-0.013451,0.020619,0.061093,0.076975,0.022552,0.069050,0.003477,...,-0.023961,0.214629,0.079856,0.026836,-0.013097,0.082758,0.000254,-0.026639,0.035447,-0.001226
8,diabetes,-0.072295,0.021619,-0.014268,0.002224,0.055424,0.057131,-0.051262,0.018889,-0.006114,...,-0.012307,0.171253,0.081780,0.015449,-0.025703,0.072341,0.087330,-0.044675,0.036224,-0.009371
9,strength,0.045398,-0.024319,-0.012122,0.019361,0.030631,0.017430,0.001756,-0.026168,-0.001378,...,-0.051959,0.171860,0.066596,0.008225,-0.051621,0.142430,0.015114,0.012061,-0.006869,0.016484


In [ ]:
train_app_usage_df['timestamp'] = pd.to_datetime(train_app_usage_df['timestamp'])
train_web_visits['timestamp'] = pd.to_datetime(train_web_visits['timestamp'])
train_claims_df['diagnosis_date'] = pd.to_datetime(train_claims_df['diagnosis_date'])
train_churn_labels_df['signup_date'] = pd.to_datetime(train_churn_labels_df['signup_date'])

app_window_end = train_app_usage_df['timestamp'].max()
web_window_end = train_web_visits['timestamp'].max()
claims_window_end = train_claims_df['diagnosis_date'].max()
observation_window_end = max(app_window_end, web_window_end, claims_window_end)
observation_window_end

### App usage feature candidates

train_app_usage_df['session_date'] = train_app_usage_df['timestamp'].dt.date
app_features = (
    train_app_usage_df
    .groupby('member_id')
    .agg(
        session_count=('timestamp', 'size'),
        active_session_days=('session_date', 'nunique'),
        first_session_ts=('timestamp', 'min'),
        last_session_ts=('timestamp', 'max')
    )
)
app_features['session_span_days'] = (app_features['last_session_ts'] - app_features['first_session_ts']).dt.days.clip(lower=0) + 1
app_features['sessions_per_active_day'] = app_features['session_count'] / app_features['active_session_days'].replace(0, pd.NA)
app_features['days_since_last_session'] = (observation_window_end - app_features['last_session_ts']).dt.days
app_features.head()

### Web visit feature candidates

train_web_visits['visit_date'] = train_web_visits['timestamp'].dt.date
web_features = (
    train_web_visits
    .groupby('member_id')
    .agg(
        web_visit_count=('timestamp', 'size'),
        unique_urls=('url', 'nunique'),
        unique_titles=('title', 'nunique'),
        web_active_days=('visit_date', 'nunique'),
        first_web_visit=('timestamp', 'min'),
        last_web_visit=('timestamp', 'max')
    )
)
web_features['days_since_last_web_visit'] = (observation_window_end - web_features['last_web_visit']).dt.days
web_features['visits_per_web_day'] = web_features['web_visit_count'] / web_features['web_active_days'].replace(0, pd.NA)
web_features.head()

### Claims-based feature candidates

train_claims_df['icd_chapter'] = train_claims_df['icd_code'].str[0]
claims_features = (
    train_claims_df
    .groupby('member_id')
    .agg(
        claim_count=('diagnosis_date', 'size'),
        unique_icd_codes=('icd_code', 'nunique'),
        last_claim_date=('diagnosis_date', 'max')
    )
)
claims_features['days_since_last_claim'] = (observation_window_end - claims_features['last_claim_date']).dt.days
chapters = train_claims_df['icd_chapter'].value_counts().index.tolist()
chapter_pivot = (
    train_claims_df[train_claims_df['icd_chapter'].isin(chapters)]
    .pivot_table(index='member_id', columns='icd_chapter', values='icd_code', aggfunc='count', fill_value=0)
)
chapter_pivot.columns = [f"claim_chapter_{col}" for col in chapter_pivot.columns]
claims_features = claims_features.join(chapter_pivot, how='left').fillna(0)
claims_features.head()

### Combine engineered features with churn labels

feature_df = (
    train_churn_labels_df[['member_id', 'signup_date', 'churn', 'outreach']]
    .merge(app_features, on='member_id', how='left')
    .merge(web_features, on='member_id', how='left')
    .merge(claims_features, on='member_id', how='left')
)
feature_df['member_tenure_days'] = (observation_window_end - feature_df['signup_date']).dt.days
numeric_cols = [col for col in feature_df.select_dtypes(include=['number']).columns if col not in ['churn']]
feature_df[numeric_cols] = feature_df[numeric_cols].fillna(0)
feature_df.head()


### Feature diagnostics vs. churn


In [20]:
feature_cols = [col for col in feature_df.columns if col not in ['member_id', 'signup_date', 'first_session_ts', 'last_session_ts', 'first_web_visit', 'last_web_visit', 'last_claim_date']]
churn_group_means = feature_df.groupby('churn')[feature_cols].mean().T
churn_group_means['mean_diff'] = churn_group_means[1] - churn_group_means[0]
churn_group_means = churn_group_means.sort_values('mean_diff', key=lambda s: s.abs(), ascending=False)
churn_group_means.head(15)


churn,0,1,mean_diff
member_tenure_days,312.914651,261.900544,-51.014107
churn,0.000000,1.000000,1.000000
session_count,9.922547,9.211776,-0.710770
unique_titles,13.179847,12.517071,-0.662776
active_session_days,7.046622,6.650668,-0.395954
session_span_days,11.569370,11.320633,-0.248736
claim_count,6.486778,6.342405,-0.144373
unique_urls,25.897105,26.037605,0.140500
web_visit_count,25.906129,26.044038,0.137909
days_since_last_session,1.022559,1.149926,0.127367


In [23]:
feature_auc = {}
for col in feature_cols:
    try:
        feature_auc[col] = roc_auc_score(feature_df['churn'], feature_df[col])
    except ValueError:
        continue
auc_summary = (
    pd.Series(feature_auc, name='roc_auc')
    .sort_values(ascending=False)
    .to_frame()
)
auc_summary.head(15)


,roc_auc
churn,1.000000
days_since_last_session,0.520413
claim_chapter_M,0.515634
claim_chapter_A,0.513092
claim_chapter_J,0.506580
claim_chapter_R,0.504915
claim_chapter_K,0.503245
days_since_last_web_visit,0.502693
visits_per_web_day,0.501337
claim_chapter_H,0.499980


In [12]:
test_churn_labels_df.head()

,member_id,signup_date,churn,outreach
0,1,2024-11-16,0,0
1,2,2024-09-12,0,1
2,3,2024-10-04,0,0
3,4,2024-02-28,0,1
4,5,2024-01-17,0,1


# Feature Engineering

## App Usage

In [25]:
train_session_count_by_member_df = train_app_usage_df[['member_id', 'timestamp']].groupby('member_id').agg('count').rename({'timestamp': 'session_count'}, axis=1)

In [26]:
train_session_count_by_member_df

,session_count
member_id,
1,6
2,10
3,9
4,10
5,10
...,...
9996,8
9997,11
9998,12


In [27]:
train_data_df = train_churn_labels_df.merge(train_session_count_by_member_df, on='member_id', how='left')

In [28]:
train_data_df = train_data_df.merge(feature_df, on='member_id', how='left')

In [26]:
train_data_df

,member_id,signup_date,churn,outreach,session_count
0,1,2024-04-12,0,0,6.0
1,2,2025-03-11,0,0,10.0
2,3,2024-09-27,0,0,9.0
3,4,2024-04-16,1,0,10.0
4,5,2024-03-12,0,1,10.0
...,...,...,...,...,...
9995,9996,2024-09-02,0,0,8.0
9996,9997,2024-02-07,0,0,11.0
9997,9998,2024-09-01,0,0,12.0
9998,9999,2024-01-21,0,0,7.0


# Model

In [ ]:
model = xgbooost